In [ ]:
# run IPYNB that imports all the relevant packages and functions
%run ../ms_packages_and_functions.ipynb

In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir   = pjoin(os.getcwd(), 'raw_data')


In [ ]:
folder_path = pjoin(output_dir,'All_Reciprocity_plotting+statistics_004')

if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    print("created directory: {}".format(folder_path))
else:
    print("All files will be saved in: {}".format(folder_path))


In [ ]:
fig_dir    = pjoin(folder_path,'figures')
if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)
    print("created directory: {}".format(fig_dir))
else:
    print("All figures will be saved in: {}".format(fig_dir))

In [ ]:
folders = ['processed_data/RS_null-metrics-reciprocity-cutoff-thresholdNodeWise-003',
           'processed_data/RJ_null-metrics-reciprocity-cutoff-thresholdNodeWise-001']

In [ ]:
# from https://stackoverflow.com/a/35094823
def autoscale_y(ax,margin=0.1):
    """This function rescales the y-axis based on the data that is visible given the current xlim of the axis.
    ax -- a matplotlib axes object
    margin -- the fraction of the total height of the y-data to pad the upper and lower ylims"""

    import numpy as np

    def get_bottom_top(line):
        xd = line.get_xdata()
        yd = line.get_ydata()
        lo,hi = ax.get_xlim()
        y_displayed = yd[((xd>lo) & (xd<hi))]
        h = np.max(y_displayed) - np.min(y_displayed)
        bot = np.min(y_displayed)-margin*h
        top = np.max(y_displayed)+margin*h
        return bot,top

    lines = ax.get_lines()
    bot,top = np.inf, -np.inf

    for line in lines:
        new_bot, new_top = get_bottom_top(line)
        if new_bot < bot: bot = new_bot
        if new_top > top: top = new_top

    ax.set_ylim(bot,top)

# Plot

In [ ]:
rec_vals={}
minima_times = {}
for x in np.arange(85,91):
    rec_vals[x] = []
    minima_times[x] = []
rec_vals

In [ ]:
# get y ranges
y_bounds = []
kernel = Gaussian1DKernel(2)

for col,folder in enumerate(folders):
    filepaths = [pjoin(folder,file) for file in os.listdir(folder) if file.startswith('norm_reciprocity-')]
    min_y = 100
    max_y = -100
    min_len = time2bin(0.75,lastBin=True)
    thresholds = np.arange(90,84,-1)
    for i,th in enumerate(thresholds):
        temp = filepaths[0].split('-')[:7]+['p'+str(th)]+filepaths[0].split('-')[8:]
        temp = ('-').join(temp)

        norm_reciprocity = loadPickle(temp)
        
        mean = np.mean(np.array([(np.array(x[:min_len])) for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]),axis=0)
        err = np.std(np.array([np.array(x[:min_len])  for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]),axis=0)
       
        y = convolve(mean,kernel)
        x = [bin2time(x,lastBin=True) for x in np.arange(min_len)]    

        #update min_y and max_y
        idx = time2bin(0.5,lastBin=True)
        if min(y[:idx])<min_y:
            min_y = min(y[:idx])
        if max(y[:idx])>max_y:
            max_y = min(y[:idx])
    y_bounds.append([min_y,max_y])
y_bounds        

In [ ]:
sns.set_context("paper")
afont = {'fontname':'Roboto','fontsize' : 16, 'ha':'center','weight':'bold'}
bfont = {'fontname':'Roboto','fontsize' : 14}
hfont = {'fontname':'Roboto','fontsize' : 16,'weight':'bold'}
cols = 2
rows = 1

subject_names = ['Monkey Rs', 'Monkey Rj']#'Monkey Bx']
fig = plt.figure(figsize=(8,4))

min_len = time2bin(0.75,lastBin=True)

row = 1
plot_th = ''

kernel = Gaussian1DKernel(2)

for col,folder in enumerate(folders):
    ax = fig.add_subplot(rows,cols,col+1)
    # look at files in folder
    filepaths = [pjoin(folder,file) for file in os.listdir(folder) if file.startswith('norm_reciprocity-')]

    # get thresholds in file
    thresholds = np.arange(90,84,-1)#np.sort([int(file.split('-')[7].split('p')[1]) for file in filepaths])
    cmap = sns.color_palette("cool",len(thresholds))

    ax.add_patch(Rectangle((0,np.mean(y_bounds[col])-0.05), 0.1, 0.1,facecolor='k',alpha=0.2))

    for i,th in enumerate(thresholds):
        temp = filepaths[0].split('-')[:7]+['p'+str(th)]+filepaths[0].split('-')[8:]
        temp = ('-').join(temp)

        norm_reciprocity = loadPickle(temp)
        
        comp_start = time2bin(0,lastBin=True)
        comp_end = time2bin(0.1,lastBin=True)
        comp_val = np.median(np.array([(np.array(x[comp_start:comp_end])) for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]))
             
        p = [stats.ttest_1samp([x[t] for d in norm_reciprocity for x in d],comp_val)[1] for t in range(min_len)]
        p_corr = multipletests(p,0.001,method='bonferroni')

        x = [bin2time(x,lastBin=True) for x in np.arange(len(p))]
    
        mean = np.mean(np.array([(np.array(x[:min_len])) for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]),axis=0)
        err = np.std(np.array([np.array(x[:min_len])  for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]),axis=0)
       
        y = convolve(mean,kernel)
        x = [bin2time(x,lastBin=True) for x in np.arange(min_len)]    
#         ax.plot(x,y,linewidth=2,label=th,c=cmap[i])

        delay_start = time2bin(0,lastBin=True)
        delay_end = time2bin(1,lastBin=True)
        minima_idx = np.argmin(y[delay_start:delay_end])+delay_start
        minima_x = np.array(x)[minima_idx]
        minima_y = np.min(y[delay_start:delay_end])        
        
        # get distribution of scores at minima
        minima_dist = [x[minima_idx] for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]
        comp_dist = [np.mean(x[comp_start:comp_end]) for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]

        if stats.ttest_ind(minima_dist,comp_dist)[1]<0.001:
            ax.plot(x,y,linewidth=2,label=th,c=cmap[i])
            plt.scatter(minima_x,y[minima_idx],c='k',s=50, marker='*',zorder=2)
            rec_vals[th].append([np.mean(minima_dist),np.std(minima_dist),np.mean(comp_dist),np.std(comp_dist)])
            minima_times[th].append(minima_x)
        else:
            ax.plot(x,y,linewidth=2,label=th,c=cmap[i],alpha=0.5)
            
        print(f'{subject_names[col]} reciprocity scores at {th} percentile : p = {stats.ttest_ind(minima_dist,comp_dist)[1]}, minima at {minima_x}')
        
        ax.text(-0.1, 1.1, string.ascii_uppercase[col], transform=ax.transAxes, 
        size=20, weight='bold')
        
        ticks = np.arange(0,0.75,0.25)
        labels = ['500 ms' if round(x,2)==0.5 else ' '  for x in ticks]
        labels[int(np.where(ticks==0)[0])] = 'Instruction'
        ax.set_xticks(ticks)
        ax.set_xticklabels(labels,**afont)
        ax.get_xticklabels()[1].set_weight("bold")
        
        
        ticks = np.arange(round(y_bounds[col][0],5)-0.005,round(y_bounds[col][0],5)+0.1,0.02)
        mid_point = np.mean(y_bounds[col])
        print(y_bounds[col],mid_point)
        
        ticks = np.arange(mid_point-0.05,mid_point+0.075,0.025)
        labels = [round(x,2)  for x in ticks]
        print(ticks)
        ax.set_yticks(ticks)
        
        ax.set_yticklabels(labels,**bfont)
        
        if col == 0:
            if i==0:
                y0 = -0.055
            ax.text(-0.05,y0+(i*0.008),str(th),c=cmap[i],**afont)
                
        plt.title(subject_names[col],**hfont)
        plt.ylabel('reciprocity (AU)',**afont)
        plt.xlim([-0.10,0.5])
        plt.ylim([min(ticks),max(ticks)])
        
        sns.despine()
#         autoscale_y(ax)

plt.tight_layout()
plt.savefig(pjoin(fig_dir,'reciprocity-letters.png'))

In [ ]:
sns.set_context("paper")
afont = {'fontname':'Roboto','fontsize' : 16, 'ha':'center','weight':'bold'}
bfont = {'fontname':'Roboto','fontsize' : 14}
hfont = {'fontname':'Roboto','fontsize' : 16,'weight':'bold'}
cols = 1
rows = 1

subject_names = ['Monkey Rs', 'Monkey Rj']#'Monkey Bx']
fig = plt.figure(figsize=(4,4))

min_len = time2bin(0.75,lastBin=True)

row = 1
plot_th = ''

kernel = Gaussian1DKernel(2)
col=1
for folder in [folders[col]]:
    ax = fig.add_subplot()
    # look at files in folder
    filepaths = [pjoin(folder,file) for file in os.listdir(folder) if file.startswith('norm_reciprocity-')]

    # get thresholds in file
#     thresholds = np.arange(90,84,-1)#np.sort([int(file.split('-')[7].split('p')[1]) for file in filepaths])
    thresholds =  [85,81,79,77,75] #[75,77,79,81,85]
    cmap = sns.color_palette("cool",len(thresholds))

    ax.add_patch(Rectangle((0,0), 0.1, -1,facecolor='k',alpha=0.2))

    for i,th in enumerate(thresholds):
        temp = filepaths[0].split('-')[:7]+['p'+str(th)]+filepaths[0].split('-')[8:]
        temp = ('-').join(temp)

        norm_reciprocity = loadPickle(temp)
        
        comp_start = time2bin(0,lastBin=True)
        comp_end = time2bin(0.1,lastBin=True)
        comp_val = np.median(np.array([(np.array(x[comp_start:comp_end])) for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]))
             
        p = [stats.ttest_1samp([x[t] for d in norm_reciprocity for x in d],comp_val)[1] for t in range(min_len)]
        p_corr = multipletests(p,0.001,method='bonferroni')

        x = [bin2time(x,lastBin=True) for x in np.arange(len(p))]
    
        mean = np.mean(np.array([(np.array(x[:min_len])) for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]),axis=0)
        err = np.std(np.array([np.array(x[:min_len])  for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]),axis=0)
       
        y = convolve(mean,kernel)
        x = [bin2time(x,lastBin=True) for x in np.arange(min_len)]    
        ax.plot(x,y,linewidth=2,label=th,c=cmap[i])

        delay_start = time2bin(0,lastBin=True)
        delay_end = time2bin(1,lastBin=True)
        minima_idx = np.argmin(y[delay_start:delay_end])+delay_start
        minima_x = np.array(x)[minima_idx]
        minima_y = np.min(y[delay_start:delay_end])        
        
        # get distribution of scores at minima
        minima_dist = [x[minima_idx] for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]
        comp_dist = [np.mean(x[comp_start:comp_end]) for d,r in enumerate(norm_reciprocity) for t,x in enumerate(r)]

        if stats.ttest_ind(minima_dist,comp_dist)[1]<0.001:
            ax.plot(x,y,linewidth=2,label=th,c=cmap[i])
            plt.scatter(minima_x,y[minima_idx],c='k',s=50, marker='*',zorder=2)
#             rec_vals[th].append([np.mean(minima_dist),np.std(minima_dist),np.mean(comp_dist),np.std(comp_dist)])
#             minima_times[th].append(minima_x)
        else:
            ax.plot(x,y,linewidth=2,label=th,c=cmap[i],alpha=0.5)
            
        print(f'{subject_names[col]} reciprocity scores at {th} percentile : p = {stats.ttest_ind(minima_dist,comp_dist)[1]}, minima at {minima_x}')
        
#         ax.text(-0.1, 1.1, string.ascii_uppercase[col], transform=ax.transAxes, 
#         size=20, weight='bold')
        
        ticks = np.arange(0,0.75,0.25)
        labels = ['500 ms' if round(x,2)==0.5 else ' '  for x in ticks]
        labels[int(np.where(ticks==0)[0])] = 'Instruction'
        ax.set_xticks(ticks)
        ax.set_xticklabels(labels,**afont)
        ax.get_xticklabels()[1].set_weight("bold")
        
        
        ticks = np.arange(round(y_bounds[col][0],5)-0.005,round(y_bounds[col][0],5)+0.1,0.02)
        mid_point = np.mean(y_bounds[col])
        print(y_bounds[col],mid_point)
        
        ticks = np.arange(mid_point-0.05,mid_point+0.150,0.025)
        labels = [round(x,2)  for x in ticks]
        print(ticks)
        ax.set_yticks(ticks)
        
        ax.set_yticklabels(labels,**bfont)
        
        if col == 1:
            if i==0:
                y0 = y_bounds[col][0]-0.005
            ax.text(0.55,y0+(i*0.015),str(th),c=cmap[i],**afont)
                
        plt.title(subject_names[col],**hfont)
        plt.ylabel('reciprocity (AU)',**afont)
        plt.xlim([-0.10,0.5])
        
        plt.ylim([min(ticks),max(ticks)])
        
        sns.despine()
#         autoscale_y(ax)

plt.tight_layout()
plt.savefig(pjoin(fig_dir,'reciprocity-supplementary.png'))